# 4 - Reflection
- We can demonstrate the core idea of a single reflection step within a LangChain chain using LCEL. This example will show how to generate an initial output, use an LLM to critique.
- While full iterative reflection often requires stateful workflows (like LangGraph), a single reflection step can be implemented in LangChain using LCEL to pass output for critique and subsequent refinement. 

In [1]:
import os
import asyncio
from langchain_openai import ChatOpenAI # Or import your preferred model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
## --- Initialize the language model ---
# Use the appropriate class and model name for your provider (e.g., ChatGoogleGenerativeAI(model="gemini-pro"))
# Setting temperature to control creativity (0.7 is a common balance)
try:
    # Example for OpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    # Example for Google (uncomment and replace if using Google)
    # from langchain_google_genai import ChatGoogleGenerativeAI
    # llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)
    print(f"Language model initialized: {llm.model_name}")

    # Create a simple prompt
    prompt = ChatPromptTemplate.from_messages([
        ("user", "Write a simple small Thought of the day!?")
    ])    
    # Create a simple chain
    chain = prompt | llm | StrOutputParser()    
    # Get the response
    response = chain.invoke({})
    print("Thought of the day:", response)

except Exception as e:
    print(f"Error initializing language model: {e}")
    print("Please ensure your API key is set correctly and the model name is valid.")
    llm = None # Set llm to None if initialization fails

Language model initialized: gpt-4o-mini
Thought of the day: "Every small step forward is a step toward your dreams—keep moving!"


In [3]:
# --- Step 1: Initial Content Generation ---
# This chain generates the initial piece of content.
generation_prompt = ChatPromptTemplate.from_messages([
   ("system", "Write a short, simple product description for a new smart coffee mug."),
   ("user", "{product_details}")
])
generation_chain = generation_prompt | llm | StrOutputParser()


In [4]:
# --- Step 2: Reflection / Critique ---
# This chain takes the initial generation and critiques it.
# The critique prompt asks the LLM to evaluate the generated description.
critique_prompt = ChatPromptTemplate.from_messages([
   ("system", """Critique the following product description.
   Focus on clarity, conciseness, and appeal to potential buyers.
   Provide specific suggestions for improvement.
   Output your critique and suggestions."""),
   ("user", "Product Description to Critique:\n{initial_description}")
])
critique_chain = critique_prompt | llm | StrOutputParser()


In [5]:
# --- Step 3: Refinement ---
# This chain takes the original product details AND the critique, and generates a refined description.
# We use RunnableParallel to pass both the original details and the critique to the refinement prompt.
refinement_prompt = ChatPromptTemplate.from_messages([
   ("system", """Based on the original product details and the following critique,
   rewrite the product description to make it more effective.
   Incorporate the suggestions from the critique.

   Original Product Details: {product_details}

   Critique: {critique}

   Refined Product Description:"""),
   ("user", "") # User input is empty as all info is in the system prompt context
])


In [6]:
# To pass both initial details and critique to the refinement step, we need to structure the input.
# We can use RunnableParallel to create a dictionary containing both.
# The input to this parallel block will be the result of the critique step.
# We also need the original product_details, so we'll pass the initial input through.
# This requires careful chaining to ensure the right information flows to the right places.

# A better way to structure this in LCEL for clarity:
# Define a chain that takes the initial product_details
# This chain will first generate the initial description,
# then critique it, and then use the original details + critique for refinement.

# Chain: Initial Details -> Generate -> Critique -> Refine
# We need to pass the initial details through the critique step to be available for refinement.
# We can use RunnableParallel to keep the initial details alongside the critique output.

# Chain:
# 1. Take initial product_details
# 2. Generate initial_description
# 3. Pass initial_description to critique_chain
# 4. Simultaneously, pass initial product_details through
# 5. Combine initial product_details and critique output for refinement_prompt
# 6. Run refinement_prompt | llm | StrOutputParser()

# Let's build this flow using LCEL:

# Step 1 & 2: Generate Initial Description
initial_generation_flow = {
   "product_details": RunnablePassthrough(), # Keep original input
   "initial_description": generation_chain # Generate initial description
}

# Step 3 & 4: Critique and Pass Through Original Details
# This block takes the output of initial_generation_flow (a dict with product_details and initial_description)
# It runs the critique_chain on the initial_description
# It also passes the product_details through
critique_and_pass_details = RunnableParallel({
   "product_details": lambda x: x['product_details'], # Pass original details through
   "initial_description": lambda x: x['initial_description'], # Also pass initial description through
   "critique": critique_chain # Run the critique chain on the initial description
})

# Step 5 & 6: Refine based on Original Details and Critique
# This block takes the output of critique_and_pass_details (a dict with product_details, initial_description, and critique)
# It formats the input for the refinement_prompt
refinement_flow = {
   "product_details": lambda x: x['product_details'],
   "critique": lambda x: x['critique'],
   "initial_description": lambda x: x['initial_description'] # Pass through if needed for context in prompt, though not strictly required by refinement_prompt here
} | refinement_prompt | llm | StrOutputParser()


# Combine the steps into a single chain
# Initial input (product_details) -> initial_generation_flow -> critique_and_pass_details -> refinement_flow
full_reflection_chain = initial_generation_flow | critique_and_pass_details | refinement_flow

In [7]:
## This code will run as a Python program,but will give error in Jupyter notebook. 
## The correct Jupyter Notebook equivalent is in the cell below.

# # --- Run the Chain ---
# async def run_reflection_example(product_details: str):
#    """Runs the LangChain reflection example with product details."""
#    if not llm:
#        print("LLM not initialized. Cannot run example.")
#        return

#    print(f"\n--- Running Reflection Example for Product: '{product_details}' ---")
#    # Invoke the chain asynchronously
#    try:
#        final_refined_description = await full_reflection_chain.ainvoke(product_details)
#        print("\n--- Final Refined Product Description ---")
#        print(final_refined_description)
#    except Exception as e:
#        print(f"\nAn error occurred during chain execution: {e}")

# # Run the example with test product details
# if __name__ == "__main__":
#    test_product_details = "A mug that keeps coffee hot and can be controlled by a smartphone app."
#    # Run the asynchronous function
#    asyncio.run(run_reflection_example(test_product_details))

#    test_product_details_2 = "A durable backpack made from recycled materials with many pockets."
#    asyncio.run(run_reflection_example(test_product_details_2))


In [8]:
# Run the example with a test topic
# pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

async def run_reflection_example(product_details: str):
   """Runs the LangChain reflection example with product details."""
   if not llm:
       print("LLM not initialized. Cannot run example.")
       return

   print(f"\n--- Running Reflection Example for Product: '{product_details}' ---")
   # Invoke the chain asynchronously
   try:
       final_refined_description = await full_reflection_chain.ainvoke(product_details)
       print("\n--- Final Refined Product Description ---")
       print(final_refined_description)
   except Exception as e:
       print(f"\nAn error occurred during chain execution: {e}")

# Then modify the running code to use await
async def main():
   test_product_details = "A mug that keeps coffee hot and can be controlled by a smartphone app."
   test_product_details = "A pen which helps you write in a beautiful handwriting."
   asyncio.run(run_reflection_example(test_product_details))

   # test_product_details_2 = "A durable backpack made from recycled materials with many pockets."
   #asyncio.run(run_reflection_example(test_product_details_2))

# Run the async function
await main()


--- Running Reflection Example for Product: 'A pen which helps you write in a beautiful handwriting.' ---

--- Final Refined Product Description ---
Revised Product Description for the Handwriting Pen:

"Transform your writing experience with our Elegant Handwriting Pen! Designed for those who desire beautiful handwriting, this pen seamlessly combines style and functionality. Its fine-tip nib allows for precise control, enabling you to create stunning letters and elegant script effortlessly.

Say goodbye to messy notes and frustrating scribbles—this pen glides smoothly across the page, making every word a joy to write. Whether you're journaling, addressing invitations, or simply expressing your thoughts, you'll relish the art of writing like never before.

Crafted with a sleek, ergonomic design, our pen fits comfortably in your hand, reducing fatigue during extended writing sessions. Plus, its vibrant ink dries quickly to prevent smudging, ensuring your work looks pristine.

Embrace t